<a href="https://colab.research.google.com/github/iambusra/complingproject/blob/main/Random%20Forest%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [305]:
!pip install pandas
!pip install xlsxwriter
!pip install openpyxl

import pandas as pd
import xlsxwriter
from google.colab import files
import io
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [294]:
# uploading the lexicon txt file which we will use for sentiment analysis via Files section on the left


In [306]:
# turning the file into a pandas data frame called lexicon

lexicon = pd.read_csv("Turkish-tr-NRC-VAD-Lexicon.txt", sep="\t", error_bad_lines=False)

In [296]:
# uploading news data via Files section on the left

In [307]:
# turning the file into a data frame called news
news = pd.read_excel("data_clean.xlsx")

# removing the first two columns, which do not contain any relevant information
news = news.drop(['Unnamed: 0', 'Haber Başlıkları'], axis = 1)
print(news)

                                               Linkler                                           Metinler
0                                    [Google Haberler]                                                NaN
1    http://www.haberinadresi.com/bursa-da-kadin-su...  [Bursa’da kimliği belirsiz bir kişi, gözüne ke...
2    http://www.pusulagazetesi.com.tr/kadin-surucu-...  Kadın sürücü kaza yaptı. Zonguldak’ta Kapuz Ma...
3    http://www.pusulagazetesi.com.tr/kadin-surucu-...  Kadın sürücü ve annesi yaralandı, kaza anı kam...
4    http://www.pusulagazetesi.com.tr/kaza-yapan-ka...  Zonguldak’ın Ereğli ilçesinde sürücünün direks...
..                                                 ...                                                ...
226  https://www.sozcu.com.tr/2021/gundem/alkollu-k...  İzmir'in Konak ilçesinde, otomobili uygulama n...
227  https://www.sozcu.com.tr/2021/gundem/seyahat-i...  Edinilen bilgilere göre, ilçe dışından gelen v...
228  https://www.sozcu.com.tr/2021/gunun-icind

In [308]:
news.describe()

,Linkler,Metinler
count,231,227
unique,222,219
top,https://www.hurriyet.com.tr/gundem/kadin-suruc...,[]
freq,2,4


In [309]:
# upload also sürücü data

# turning the file into a data frame called news
news_sürücü = pd.read_excel("/content/data_surucu.xlsx")

# removing the first two columns, which do not contain any relevant information
news_sürücü = news_sürücü.drop(['Unnamed: 0', 'Haber Başlıkları'], axis = 1)
print(news_sürücü)

                                              Linkler                                           Metinler
0                                   [Google Haberler]                                                NaN
1   http://www.adayorum.com/alkolltrafige-kapali-y...  Sakarya’nın Sapanca ilçesinde alkollü sürücü i...
2   http://www.boluolay.com/bolu/dur-ihtarina-uyma...  Polisin dur ihtarına uymayarak kaçmaya başlaya...
3   https://www.sakaryadanhaber.com/haber/7262954/...  Kocaeli’nin Gebze ilçesinde 8 yaşındaki çocuk,...
4   https://www.golcukgundem.com/haber/7259308/alk...  Alkollü olduğu iddia edilen sürücü aşırı hız y...
..                                                ...                                                ...
93  https://tr.sputniknews.com/abd/202105271044596...  ABD’de geçtiğimiz pazar günü önce telefonda po...
94  https://shiftdelete.net/nvidia-bazi-ekran-kart...  SDN Network, SDN Teknoloji Haberleri, Mediatre...
95  https://www.haberturk.com/kayseri-haberleri/87...  

In [310]:
news_sürücü["label"] = "neutral"
news["label"] = "biased"
news = news.append(news_sürücü)

In [311]:
news.describe()

,Linkler,Metinler,label
count,329,321,329
unique,318,313,2
top,https://www.hurriyet.com.tr/gundem/kadin-suruc...,[],biased
freq,2,4,231


In [315]:
# there are duplicates in Linkler and Metinler column. Let's remove duplicates from the data
news = news.drop_duplicates(subset='Linkler')
news = news.drop_duplicates(subset='Metinler')

# Let's also drop NaN rows
news = news.dropna()
news.describe()

,Linkler,Metinler,label
count,305,305,305
unique,305,305,2
top,https://www.milliyet.com.tr/milliyet-tv/kadin-...,"Kadın sürücü, çarptığı otomobile takla attırdı...",biased
freq,1,1,212


In [316]:
# preprocessing function that we have written in the class.
# We are using Turkish stop words this time since our data is in Turkish.

import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('turkish'))

def PreprocessNews(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokenized_text = nltk.word_tokenize(text)
  clean_text = [word for word in tokenized_text if word not in stopwords ]
  return(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [317]:
# preprocessing our news bodies
news['Metinler'] = news.apply(lambda x: PreprocessNews(x['Metinler']), axis=1)

**TOKENIZER NEEDED HERE**

# Assigning Sentiment Values to Tokens in the News Body

In [318]:
# create empty lists for saving sentiment values in the loop below
A = []
D = []
V =[]

# create variables with type int and value 0 so that we can add the sentiment values
# for each token to them and return the total sentiment values of a news body in the end.


# Look at the news body one by one (one row at a time).
# Then for the news body in that row, find sentiment value for each word.
# Add them up until moving onto the next news body in the next row.
for metin in news['Metinler']:
  arousal = 0
  dominance = 0
  valence = 0
  for word in metin:
    if word in lexicon["Turkish-tr"].tolist():
      arousal += lexicon["Arousal"][lexicon["Turkish-tr"].tolist().index(word)]
      dominance += lexicon["Dominance"][lexicon["Turkish-tr"].tolist().index(word)]
      valence += lexicon["Valence"][lexicon["Turkish-tr"].tolist().index(word)]

  A.append(arousal)
  D.append(dominance)
  V.append(valence)



print(A)
print(D)
print(V)

[31.07000000000001, 12.541, 18.584, 18.637, 0.292, 0, 66.216, 0, 52.70599999999998, 137.38899999999998, 10.834000000000001, 0.588, 0.32799999999999996, 13.437999999999999, 41.70000000000002, 29.207, 4.5329999999999995, 49.108999999999995, 7.458999999999999, 23.838000000000005, 23.034, 19.987, 18.975999999999996, 18.836, 40.48500000000001, 16.122999999999998, 14.297, 55.43600000000001, 30.901000000000007, 23.658, 8.236, 11.995999999999997, 27.623, 15.634000000000002, 88.61999999999999, 58.688999999999986, 48.644999999999975, 76.469, 163.2339999999999, 63.855999999999995, 20.749999999999993, 22.996999999999993, 40.208000000000006, 39.52799999999999, 78.35400000000004, 27.650999999999996, 48.73700000000001, 6.069, 28.273999999999994, 19.665, 13.865, 21.628, 4.198, 3.4059999999999997, 2.829, 14.87, 20.559000000000005, 21.690999999999992, 14.084999999999997, 5.064, 25.735999999999997, 8.546, 16.900000000000002, 37.067, 17.465, 29.756000000000007, 39.55800000000004, 30.67800000000002, 15.042

**NOT SURE IF WE NEED TO TURN SENTIMENT VALUES INTO FLOATS RATHER THAN LEAVING THEM AS LISTS**

In [319]:
# Check if the number of rows in the news match the number of items in the lists A, D, and V.
print(len(A))
print(len(D))
print(len(V))

# 305 rows as expected. Great!

305
305
305


In [320]:
# Let's append these lists with sentiment values to news data frame.
news["Arousal"] = A
news["Dominance"] = D
news["Valence"] = V
news.head()

,Linkler,Metinler,label,Arousal,Dominance,Valence
1,http://www.haberinadresi.com/bursa-da-kadin-su...,"[bursa, ’, kimliği, belirsiz, bir, kişi, gözün...",biased,31.070,32.668,36.459
2,http://www.pusulagazetesi.com.tr/kadin-surucu-...,"[kadın, sürücü, kaza, yaptı, zonguldak, ’, ta,...",biased,12.541,11.451,12.472
3,http://www.pusulagazetesi.com.tr/kadin-surucu-...,"[kadın, sürücü, annesi, yaralandı, kaza, anı, ...",biased,18.584,19.859,22.878
4,http://www.pusulagazetesi.com.tr/kaza-yapan-ka...,"[zonguldak, ’, ın, ereğli, ilçesinde, sürücünü...",biased,18.637,17.329,16.638
5,https://basakgazetesi.com/haber/ozelpark-halin...,"[tanıtımın, önemi, ülkenizi, sevinbedava, rekl...",biased,0.292,0.370,0.542


In [321]:
listed_tokens = []
for metin in news['Metinler']:
  news_listed_tokens = 0
  for word in metin:
    if word in lexicon["Turkish-tr"].tolist():
      news_listed_tokens += 1
  listed_tokens.append(news_listed_tokens)
print(listed_tokens)

[65, 23, 38, 34, 1, 0, 134, 0, 113, 298, 21, 2, 1, 27, 86, 58, 10, 102, 17, 46, 46, 40, 38, 38, 95, 29, 30, 113, 63, 44, 17, 24, 64, 32, 193, 121, 106, 159, 342, 131, 43, 48, 88, 88, 163, 54, 105, 13, 56, 41, 29, 43, 8, 8, 6, 28, 37, 48, 28, 11, 52, 20, 31, 79, 36, 63, 90, 64, 30, 45, 30, 26, 25, 65, 13, 10, 56, 108, 21, 73, 24, 11, 83, 37, 47, 20, 13, 42, 8, 24, 76, 25, 55, 32, 78, 88, 62, 32, 82, 20, 54, 39, 44, 33, 35, 38, 40, 48, 36, 41, 41, 40, 34, 52, 55, 93, 60, 48, 33, 15, 56, 54, 52, 28, 32, 59, 24, 55, 75, 51, 84, 37, 28, 28, 29, 59, 31, 35, 38, 36, 49, 23, 40, 7, 31, 45, 24, 11, 41, 37, 13, 51, 109, 137, 104, 44, 50, 55, 36, 37, 43, 82, 87, 84, 116, 61, 72, 52, 62, 38, 30, 49, 73, 106, 109, 65, 103, 47, 61, 67, 35, 14, 50, 27, 34, 48, 32, 49, 37, 25, 27, 43, 43, 144, 23, 95, 24, 19, 30, 26, 33, 27, 23, 131, 60, 42, 44, 51, 86, 40, 73, 26, 89, 72, 101, 104, 37, 9, 26, 21, 24, 27, 24, 13, 89, 36, 16, 52, 101, 27, 31, 46, 35, 23, 12, 21, 134, 24, 134, 21, 43, 23, 28, 32, 42, 18

In [322]:
token_df = pd.DataFrame(list(zip(news["Arousal"], news["Dominance"], news["Valence"], listed_tokens)), columns = ["Arousal", "Dominance", "Valence", "Listed Tokens"])

normalized_arousal = []
normalized_dominance = []
normalized_valence = []

for i in range(len(news["Arousal"])):
  normalized_arousal.append(token_df["Arousal"][i]/token_df["Listed Tokens"][i])
  normalized_dominance.append(token_df["Dominance"][i]/token_df["Listed Tokens"][i])
  normalized_valence.append(token_df["Valence"][i]/token_df["Listed Tokens"][i])

print(normalized_arousal)
print(normalized_dominance)
print(normalized_valence)

[0.47800000000000015, 0.5452608695652174, 0.48905263157894735, 0.5481470588235294, 0.292, nan, 0.4941492537313432, nan, 0.4664247787610618, 0.4610369127516778, 0.515904761904762, 0.294, 0.32799999999999996, 0.49770370370370365, 0.48488372093023274, 0.5035689655172414, 0.4532999999999999, 0.48146078431372547, 0.4387647058823529, 0.518217391304348, 0.5007391304347826, 0.499675, 0.49936842105263146, 0.4956842105263157, 0.4261578947368422, 0.5559655172413792, 0.4765666666666667, 0.4905840707964602, 0.4904920634920636, 0.5376818181818183, 0.48447058823529415, 0.4998333333333332, 0.431609375, 0.48856250000000007, 0.4591709844559585, 0.48503305785123957, 0.4589150943396224, 0.48093710691823893, 0.4772923976608184, 0.48745038167938926, 0.4825581395348836, 0.4791041666666665, 0.456909090909091, 0.44918181818181807, 0.48069938650306776, 0.5120555555555555, 0.46416190476190483, 0.46684615384615386, 0.504892857142857, 0.4796341463414634, 0.4781034482758621, 0.5029767441860465, 0.52475, 0.425749999

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [323]:
# Let's append these lists with sentiment values to news data frame.
news["Normalized Arousal"] = normalized_arousal
news["Normalized Dominance"] = normalized_dominance
news["Normalized Valence"] = normalized_valence
news.head()

,Linkler,Metinler,label,Arousal,Dominance,Valence,Normalized Arousal,Normalized Dominance,Normalized Valence
1,http://www.haberinadresi.com/bursa-da-kadin-su...,"[bursa, ’, kimliği, belirsiz, bir, kişi, gözün...",biased,31.070,32.668,36.459,0.478000,0.502585,0.560908
2,http://www.pusulagazetesi.com.tr/kadin-surucu-...,"[kadın, sürücü, kaza, yaptı, zonguldak, ’, ta,...",biased,12.541,11.451,12.472,0.545261,0.497870,0.542261
3,http://www.pusulagazetesi.com.tr/kadin-surucu-...,"[kadın, sürücü, annesi, yaralandı, kaza, anı, ...",biased,18.584,19.859,22.878,0.489053,0.522605,0.602053
4,http://www.pusulagazetesi.com.tr/kaza-yapan-ka...,"[zonguldak, ’, ın, ereğli, ilçesinde, sürücünü...",biased,18.637,17.329,16.638,0.548147,0.509676,0.489353
5,https://basakgazetesi.com/haber/ozelpark-halin...,"[tanıtımın, önemi, ülkenizi, sevinbedava, rekl...",biased,0.292,0.370,0.542,0.292000,0.370000,0.542000


In [324]:
# saving the data frame with sentiment values
writer = pd.ExcelWriter('Sentiment_data_for_random_forest.xlsx')
news.to_excel(writer)
writer.save()

In [325]:
# let's check if there is any Null slot in the data, especially in the new columns since we have dropped NaNs before already.
news = news[news['Normalized Arousal'].notna()]
print("Sum of NULL values in each column. ")
print(news.isnull().sum())

Sum of NULL values in each column. 
Linkler                 0
Metinler                0
label                   0
Arousal                 0
Dominance               0
Valence                 0
Normalized Arousal      2
Normalized Dominance    2
Normalized Valence      2
dtype: int64


In [326]:
# we have two Null values in the new columns. We drop them and check again if it worked.
news = news[news['Normalized Arousal'].notna()]
print("Sum of NULL values in each column. ")
print(news.isnull().sum())

Sum of NULL values in each column. 
Linkler                 0
Metinler                0
label                   0
Arousal                 0
Dominance               0
Valence                 0
Normalized Arousal      0
Normalized Dominance    0
Normalized Valence      0
dtype: int64


In [327]:
news.describe #303 rows, 9 columns

<bound method NDFrame.describe of                                               Linkler  ... Normalized Valence
1   http://www.haberinadresi.com/bursa-da-kadin-su...  ...           0.560908
2   http://www.pusulagazetesi.com.tr/kadin-surucu-...  ...           0.542261
3   http://www.pusulagazetesi.com.tr/kadin-surucu-...  ...           0.602053
4   http://www.pusulagazetesi.com.tr/kaza-yapan-ka...  ...           0.489353
5   https://basakgazetesi.com/haber/ozelpark-halin...  ...           0.542000
..                                                ...  ...                ...
93  https://tr.sputniknews.com/abd/202105271044596...  ...           0.543231
94  https://shiftdelete.net/nvidia-bazi-ekran-kart...  ...           0.615722
95  https://www.haberturk.com/kayseri-haberleri/87...  ...           0.593815
96  https://www.milliyet.com.tr/gundem/kisitlamayi...  ...           0.533054
97  https://www.milliyet.com.tr/gundem/surucu-bir-...  ...           0.564143

[303 rows x 9 columns]>

# Random Forest Classifier


In [328]:
# For the news data frame, we select only the columns we need for building a classifier.
# X includes our 3 predictors and y includes the predicted value (the category of the news as biased or neutral)
X = news[["Normalized Arousal", "Normalized Dominance", "Normalized Valence"]]
y = news["label"]

In [332]:
# We encode the outcome (biased/neutral) by numerical values for model's use
labelencoder_y = LabelEncoder()

# fit and transform the dependent variable (our outcome) y
y = labelencoder_y.fit_transform(y)

In [333]:
# Splitting the data into test and train dataset such that 70% of the data will be training dataset and 30%  test dataset
X_train, X_test, y_train, y_test = train_test_split(
			X, y, test_size = 0.3, random_state = 0)

In [334]:
# Using the random forest classifier for the prediction
classifier=RandomForestClassifier()
classifier=classifier.fit(X_train,y_train)
predicted=classifier.predict(X_test)

In [335]:
# printing the results
print ('Confusion Matrix :')
print(confusion_matrix(y_test, predicted))
print ('Accuracy Score :',accuracy_score(y_test, predicted))
print ('Report : ')
print (classification_report(y_test, predicted))

Confusion Matrix :
[[53  6]
 [24  8]]
Accuracy Score : 0.6703296703296703
Report : 
              precision    recall  f1-score   support

           0       0.69      0.90      0.78        59
           1       0.57      0.25      0.35        32

    accuracy                           0.67        91
   macro avg       0.63      0.57      0.56        91
weighted avg       0.65      0.67      0.63        91

